In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('core-nomad-381608-a81e95959692.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/13"
pmc = "01071010" # 店舗ID
SPREADSHEET_KEY = '1XIXPUuDpaITGNkSl2kw3d7kZ1euGLF8yYOTzWRFsC50' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 385 # スロットの1台目
wait = 0.01

user_id = "g84909252+002@gmail.com"
user_pw = "sitega21"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777_2'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "125":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "125":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/13', '025765', '1', '32308.0', '-902.0', '35', '17']
['08/13', '025765', '2', '25128.0', '-902.0', '28', '9']
['08/13', '025765', '3', '19487.0', '-6319.0', '14', '8']
['08/13', '025765', '4', '17436.0', '-2707.0', '16', '5']
['08/13', '025317', '5', '17692.0', '6621.0', '35', '9']
['08/13', '025317', '6', '32564.0', '-3610.0', '31', '11']
['08/13', '025877', '7', '17179.0', '6320.0', '27', '6']
['08/13', '025877', '8', '26923.0', '-2106.0', '26', '9']
['08/13', '025877', '9', '39231.0', '-1203.0', '42', '17']
['08/13', '025877', '10', '24359.0', '1506.0', '26', '10']
['08/13', '025877', '11', '38974.0', '603.0', '43', '14']
['08/13', '025877', '12', '16667.0', '2408.0', '21', '7']
['08/13', '025877', '13', '34872.0', '6320.0', '48', '22']
['08/13', '025877', '14', '13077.0', '-4513.0', '9', '7']
['08/13', '025877', '15', '26410.0', '2408.0', '31', '14']
['08/13', '025877', '16', '37436.0', '-8425.0', '28', '13']
['08/13', '025521', '17', '24359.0', '1205.0', '25', '13']
['08/13',

['08/13', '025665', '143', '27179.0', '9631.0', '21', '4']
['08/13', '025665', '144', '33846.0', '9029.0', '24', '3']
['08/13', '025670', '145', '26154.0', '-14444.0', '10', '4']
['08/13', '025483', '146', '12051.0', '11737.0', '18', '3']
['08/13', '025750', '147', '27436.0', '-11133.0', '29', '3']
['08/13', '025837', '148', '8718.0', '8126.0', '8', '2']
['08/13', '025667', '149', '6410.0', '-2406.0', '4', '3']
['08/13', '025519', '150', '23077.0', '-2406.0', '11', '2']
['08/13', '025825', '151', '1795.0', '-1504.0', '0', '0']
['08/13', '025826', '152', '7436.0', '-3610.0', '2', '2']
['08/13', '025573', '153', '0', '0', '1', '1']
['08/13', '025829', '154', '42821.0', '-25578.0', '6', '2']
['08/13', '025830', '155', '3590.0', '-2406.0', '0', '0']
['08/13', '025825', '156', '13333.0', '2408.0', '9', '1']
['08/13', '025645', '157', '13590.0', '3311.0', '9', '2']
['08/13', '025731', '158', '10000.0', '-1203.0', '11', '3']
['08/13', '025544', '159', '12564.0', '-6920.0', '8', '4']
['08/13',

['08/13', '025837', '289', '21026.0', '-2707.0', '9', '1']
['08/13', '025814', '290', '0', '0', '0', '0']
['08/13', '025764', '291', '18205.0', '-11735.0', '2', '2']
['08/13', '025830', '292', '0', '0', '0', '0']
['08/13', '025827', '293', '0', '0', '0', '0']
['08/13', '025800', '294', '32308.0', '-25879.0', '2', '2']
['08/13', '025800', '295', '3077.0', '-2707.0', '0', '0']
['08/13', '025800', '296', '0', '0', '0', '0']
['08/13', '025849', '297', '2564.0', '-2406.0', '0', '0']
['08/13', '025849', '298', '8718.0', '12640.0', '13', '2']
['08/13', '025849', '299', '2564.0', '-1203.0', '1', '1']
['08/13', '025849', '300', '3846.0', '8728.0', '8', '2']
['08/13', '025825', '301', '13333.0', '-7221.0', '4', '4']
['08/13', '025825', '302', '0', '0', '0', '0']
['08/13', '025825', '303', '3590.0', '-2406.0', '1', '1']
['08/13', '025825', '304', '513.0', '-902.0', '0', '0']
['08/13', '025489', '305', '13077.0', '2107.0', '7', '1']
['08/13', '025489', '306', '5897.0', '4214.0', '5', '1']
['08/13'

['08/13', '120122', '429', '2570', '-370.0', '23', '7', '--']
['08/13', '120122', '430', '4127', '-93.0', '44', '12', '--']
['08/13', '120122', '431', '2766', '-3056.0', '8', '3', '--']
['08/13', '120122', '432', '6848', '-139.0', '72', '20', '--']
['08/13', '120010', '433', '1617', '-556.0', '4', '3', '--']
['08/13', '120010', '434', '7908', '-185.0', '29', '29', '--']
['08/13', '120010', '435', '5148', '556.0', '21', '12', '--']
['08/13', '120010', '436', '2959', '-139.0', '12', '8', '--']
['08/13', '120010', '437', '3773', '278.0', '16', '7', '--']
['08/13', '120010', '438', '2349', '-1157.0', '5', '6', '--']
['08/13', '120010', '439', '7103', '1389.0', '31', '23', '--']
['08/13', '120010', '440', '1758', '-324.0', '6', '2', '--']
['08/13', '120010', '441', '6290', '-2454.0', '15', '13', '--']
['08/13', '120010', '442', '1905', '-880.0', '5', '3', '--']
['08/13', '120010', '443', '2985', '-741.0', '9', '5', '--']
['08/13', '120010', '444', '2089', '-1019.0', '5', '4', '--']
['08/13'

['08/13', '120104', '564', '3054', '-278.0', '14', '6', '--']
['08/13', '120127', '565', '3953', '-509.0', '15', '2', '--']
['08/13', '120076', '566', '1303', '-185.0', '10', '0', '--']
['08/13', '119897', '567', '1574', '185.0', '16', '13', '--']
['08/13', '120070', '568', '1053', '-1296.0', '2', '2', '--']
['08/13', '120109', '569', '2472', '93.0', '40', '5', '--']
['08/13', '119860', '570', '3917', '-694.0', '26', '26', '--']
['08/13', '120127', '571', '5180', '-2546.0', '10', '0', '--']
['08/13', '120112', '572', '4038', '324.0', '26', '16', '--']
['08/13', '120121', '573', '1988', '-1898.0', '6', '3', '--']
['08/13', '120091', '574', '2380', '648.0', '56', '17', '--']
['08/13', '119966', '575', '1571', '-694.0', '8', '4', '--']
['08/13', '120032', '576', '2280', '3843.0', '66', '6', '--']


{'spreadsheetId': '1XIXPUuDpaITGNkSl2kw3d7kZ1euGLF8yYOTzWRFsC50',
 'updates': {'spreadsheetId': '1XIXPUuDpaITGNkSl2kw3d7kZ1euGLF8yYOTzWRFsC50',
  'updatedRange': 'S5RAW!F3842:M3873',
  'updatedRows': 32,
  'updatedColumns': 8,
  'updatedCells': 256}}

In [6]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "125":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "125":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/12', '025765', '1', '32308.0', '3612.0', '42', '14']
['08/12', '025765', '2', '26410.0', '7825.0', '41', '13']
['08/12', '025765', '3', '28462.0', '3010.0', '38', '13']
['08/12', '025765', '4', '36923.0', '-3309.0', '36', '15']
['08/12', '025317', '5', '7436.0', '-1203.0', '7', '4']
['08/12', '025317', '6', '26154.0', '904.0', '33', '12']
['08/12', '025877', '7', '27179.0', '20765.0', '57', '12']
['08/12', '025877', '8', '46154.0', '-5717.0', '44', '20']
['08/12', '025877', '9', '33846.0', '-6319.0', '26', '12']
['08/12', '025877', '10', '42051.0', '-4814.0', '38', '15']
['08/12', '025877', '11', '34615.0', '4816.0', '43', '16']
['08/12', '025877', '12', '30513.0', '5418.0', '35', '11']
['08/12', '025877', '13', '41026.0', '3612.0', '47', '15']
['08/12', '025877', '14', '35641.0', '5117.0', '43', '14']
['08/12', '025877', '15', '25128.0', '-5115.0', '20', '8']
['08/12', '025877', '16', '42564.0', '-1805.0', '40', '17']
['08/12', '025521', '17', '33590.0', '-8124.0', '21', '11']
['

['08/12', '025665', '141', '48462.0', '-4814.0', '25', '6']
['08/12', '025665', '142', '35128.0', '-1203.0', '19', '4']
['08/12', '025665', '143', '44872.0', '-17453.0', '14', '4']
['08/12', '025665', '144', '45897.0', '-3309.0', '22', '3']
['08/12', '025670', '145', '20000.0', '-6619.0', '13', '4']
['08/12', '025483', '146', '11282.0', '8126.0', '16', '2']
['08/12', '025750', '147', '8462.0', '-6018.0', '6', '1']
['08/12', '025837', '148', '8718.0', '-4513.0', '2', '1']
['08/12', '025667', '149', '8205.0', '-6018.0', '1', '1']
['08/12', '025519', '150', '31282.0', '-11735.0', '11', '5']
['08/12', '025825', '151', '7436.0', '10533.0', '11', '1']
['08/12', '025826', '152', '20769.0', '-902.0', '12', '2']
['08/12', '025573', '153', '14615.0', '3913.0', '14', '4']
['08/12', '025829', '154', '16667.0', '-11735.0', '1', '1']
['08/12', '025830', '155', '15641.0', '-3610.0', '8', '4']
['08/12', '025825', '156', '6410.0', '-4513.0', '1', '1']
['08/12', '025645', '157', '14359.0', '-10832.0', '

['08/12', '025850', '283', '513.0', '-902.0', '0', '0']
['08/12', '025829', '284', '11795.0', '2107.0', '8', '1']
['08/12', '025792', '285', '0', '0', '0', '0']
['08/12', '025585', '286', '8718.0', '-6619.0', '1', '1']
['08/12', '025806', '287', '6667.0', '-5115.0', '0', '0']
['08/12', '025645', '288', '18462.0', '-14744.0', '0', '0']
['08/12', '025837', '289', '6923.0', '8126.0', '8', '1']
['08/12', '025814', '290', '769.0', '-1504.0', '0', '0']
['08/12', '025764', '291', '2308.0', '-902.0', '1', '1']
['08/12', '025830', '292', '0', '0', '0', '0']
['08/12', '025827', '293', '8462.0', '-1805.0', '4', '1']
['08/12', '025800', '294', '23846.0', '-13240.0', '7', '4']
['08/12', '025800', '295', '0', '0', '0', '0']
['08/12', '025800', '296', '9744.0', '-8124.0', '0', '0']
['08/12', '025849', '297', '1795.0', '3010.0', '3', '1']
['08/12', '025849', '298', '9231.0', '1205.0', '6', '1']
['08/12', '025849', '299', '0', '0', '0', '0']
['08/12', '025849', '300', '1538.0', '1806.0', '2', '1']
['08

['08/12', '120145', '424', '1793', '-370.0', '24', '3', '--']
['08/12', '120122', '425', '3448', '1111.0', '45', '8', '--']
['08/12', '120122', '426', '1001', '-509.0', '7', '2', '--']
['08/12', '120122', '427', '2351', '-463.0', '25', '3', '--']
['08/12', '120122', '428', '4652', '-185.0', '52', '13', '--']
['08/12', '120122', '429', '3401', '417.0', '39', '11', '--']
['08/12', '120122', '430', '4453', '-3380.0', '23', '9', '--']
['08/12', '120122', '431', '1780', '-972.0', '12', '4', '--']
['08/12', '120122', '432', '4289', '-3380.0', '20', '9', '--']
['08/12', '120010', '433', '5338', '-602.0', '17', '15', '--']
['08/12', '120010', '434', '2623', '-509.0', '7', '7', '--']
['08/12', '120010', '435', '3297', '-833.0', '11', '5', '--']
['08/12', '120010', '436', '7817', '1435.0', '32', '27', '--']
['08/12', '120010', '437', '4507', '1528.0', '22', '13', '--']
['08/12', '120010', '438', '4200', '139.0', '15', '13', '--']
['08/12', '120010', '439', '3242', '-231.0', '11', '7', '--']
['08

['08/12', '120077', '557', '2371', '-2037.0', '15', '4', '--']
['08/12', '120077', '558', '1434', '-139.0', '16', '11', '--']
['08/12', '120077', '559', '1677', '2269.0', '32', '21', '--']
['08/12', '120077', '560', '2044', '1991.0', '45', '24', '--']
['08/12', '120104', '561', '2788', '-1296.0', '13', '3', '--']
['08/12', '120104', '562', '1396', '231.0', '10', '4', '--']
['08/12', '120104', '563', '974', '926.0', '6', '3', '--']
['08/12', '120104', '564', '3818', '1435.0', '28', '11', '--']
['08/12', '120127', '565', '1539', '3935.0', '10', '1', '--']
['08/12', '120076', '566', '1959', '-1111.0', '10', '0', '--']
['08/12', '119897', '567', '2903', '1019.0', '33', '33', '--']
['08/12', '120070', '568', '1379', '463.0', '26', '5', '--']
['08/12', '120109', '569', '1155', '-1111.0', '9', '3', '--']
['08/12', '119860', '570', '4198', '-2407.0', '18', '18', '--']
['08/12', '120127', '571', '3708', '972.0', '17', '2', '--']
['08/12', '120112', '572', '3335', '-463.0', '20', '10', '--']
['0

{'spreadsheetId': '1XIXPUuDpaITGNkSl2kw3d7kZ1euGLF8yYOTzWRFsC50',
 'updates': {'spreadsheetId': '1XIXPUuDpaITGNkSl2kw3d7kZ1euGLF8yYOTzWRFsC50',
  'updatedRange': 'S5RAW!F3810:M3841',
  'updatedRows': 32,
  'updatedColumns': 8,
  'updatedCells': 256}}